# Project 2 - Machine Learning, Fall 2022

Deadline: 11th of December 2022, 23:59

To do this project you have to complete this Jupyter notebook and send it via Discord.

The total number of points allocated for this project is 10.

You will need the following modules to solve the tasks:

In [465]:
import math
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from itertools import product

In [466]:
penguin_dataset = pd.read_csv("data/penguins_filtered.csv")

penguin_dataset = penguin_dataset.replace({
    "Adelie": 1,
    "Chinstrap" : 2,
    "Gentoo": 3,
    "male" : 0,
    "female" : 1,
    "Biscoe" : 0,
    "Dream" : 1,
    "Torgersen" : 2})

discrete_penguin_dataset = penguin_dataset[["sex", "island", "species"]]
print(penguin_dataset)
print(discrete_penguin_dataset.head())

     sex  island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0      0       2            39.1           18.7              181.0   
1      1       2            39.5           17.4              186.0   
2      1       2            40.3           18.0              195.0   
3      1       2            36.7           19.3              193.0   
4      0       2            39.3           20.6              190.0   
..   ...     ...             ...            ...                ...   
328    0       1            55.8           19.8              207.0   
329    1       1            43.5           18.1              202.0   
330    0       1            49.6           18.2              193.0   
331    0       1            50.8           19.0              210.0   
332    1       1            50.2           18.7              198.0   

     body_mass_g  species  
0         3750.0        1  
1         3800.0        1  
2         3250.0        1  
3         3450.0        1  
4         3650.0   

## I. Naive and Joint Bayes (3.5 points; 0.15 bonus per week)

1. Calculate the prior probabilities for the target feature. Transform them by applying the natural logarithm. 

In [467]:
# solution here
species, species_nr = np.unique(penguin_dataset["species"], return_counts=True)
p_species = []
for specie_nr in species_nr:
    p_species.append(specie_nr / sum(species_nr))

print(p_species)

for i in range(len(p_species)):
    p_species[i] = np.log(p_species[i])

print(p_species)

[0.43843843843843844, 0.2042042042042042, 0.35735735735735735]
[-0.8245358682721073, -1.5886347848043372, -1.0290189968689145]


2. Write the formulas used to calculate the maximum aposteriori probability using Naive Bayes and Joint Bayes. Use the names of the variables from the discrete penguin dataset.

*Answer here*:

In [468]:
#NAIVE BAYES: v_MAP = argmax P(a_i|v_j)P(v_j), where a_i is an attribute and v_j is target atrribute value
#JOINT BAYES: v_MAP = argmax P(a_1,a_2,...,a_n|v_j)P(v_j)

#naive bayes
#P(sex=x|species=1)P(island=y|species=1)P(species=1), x in {0,1}, y in {0,1,2}

#joint bayes
#P(sex=x,island=y|species=1)P(species=1), x in {0,1}, y in {0,1,2}

3. Find and calculate the logarithm of all the conditional probabilities (also names likelihoods) used to predict the label for the instance `{"sex" : 1, "island" : 2}` in Naive Bayes.

In [469]:
# solution here
#P(sex=1|species=1),P(island=2|species=1)
#P(sex=1|species=2),P(island=2|species=2)
#P(sex=1|species=3),P(island=2|species=3)

def calculate_conditional_probability(dataset, feature, target_feature, alpha, print_res = True):
    feature_vals, nr_feature_vals = np.unique(dataset[feature], return_counts=True)
    tf_name = list(target_feature.keys())[0]
    tf_val = target_feature[tf_name]
    dict = {}
    for feature_val in feature_vals:
        temp_dataset = dataset[dataset[feature] == feature_val]
        target_feature_vals, nr_target_feature_vals = np.unique(temp_dataset[tf_name], return_counts=True)
        #print(target_feature_vals, nr_target_feature_vals)
        i_tf_val = -1
        #print(target_feature_vals, nr_target_feature_vals)
        for i in range(len(target_feature_vals)):
            if target_feature_vals[i] == tf_val:
                i_tf_val = i
                break
        #print(i_tf_val, tf_val)
        #print(feature_val, nr_target_feature_vals[i_tf_val])
        if alpha==0:
            if i_tf_val == -1: #if target feature is not found
                p_tf_val = 0
            else:
                p_tf_val = nr_target_feature_vals[i_tf_val] / sum(nr_target_feature_vals)
        else:
            laplace_denom = len(np.unique(penguin_dataset[tf_name]))
            if i_tf_val == -1: #if target feature is not found
                p_tf_val = 1 / (sum(nr_target_feature_vals) + laplace_denom)
            else:
                p_tf_val = (nr_target_feature_vals[i_tf_val] + alpha) / (sum(nr_target_feature_vals) + laplace_denom)
            #print(nr_target_feature_vals[i_tf_val] + alpha, sum(nr_target_feature_vals) + laplace_denom)
        res_string = "P(" + tf_name + "=" + str(tf_val) + "|" + feature + "=" + str(feature_val) + ") = " + str(p_tf_val)
        if print_res:
            print(res_string)
            print("ln(" + res_string + ") = " + str(np.log(p_tf_val)))
            print()
        dict[feature_val] = p_tf_val
    return dict


calculate_conditional_probability(penguin_dataset,"species",{ "sex" : 1},0)
calculate_conditional_probability(penguin_dataset,"species",{ "island" : 2},0)

P(sex=1|species=1) = 0.5
ln(P(sex=1|species=1) = 0.5) = -0.6931471805599453

P(sex=1|species=2) = 0.5
ln(P(sex=1|species=2) = 0.5) = -0.6931471805599453

P(sex=1|species=3) = 0.48739495798319327
ln(P(sex=1|species=3) = 0.48739495798319327) = -0.7186804825651101

P(island=2|species=1) = 0.3219178082191781
ln(P(island=2|species=1) = 0.3219178082191781) = -1.133459019998278

P(island=2|species=2) = 0
ln(P(island=2|species=2) = 0) = -inf

P(island=2|species=3) = 0
ln(P(island=2|species=3) = 0) = -inf



C:\Users\godar\AppData\Local\Temp\ipykernel_5528\2751706160.py:38: RuntimeWarning: divide by zero encountered in log
  print("ln(" + res_string + ") = " + str(np.log(p_tf_val)))


{1: 0.3219178082191781, 2: 0, 3: 0}

4. Why does the results contain infinity? Fix the calculation by using the Laplace Smoothing with `alpha = 1`.

In [470]:
# solution here
# the results contain infinity because the posterior probability of some values is 0 and ln(0) is -infinity
calculate_conditional_probability(penguin_dataset,"species",{ "sex" : 1},1)
calculate_conditional_probability(penguin_dataset,"species",{ "island" : 2},1)

P(sex=1|species=1) = 0.5
ln(P(sex=1|species=1) = 0.5) = -0.6931471805599453

P(sex=1|species=2) = 0.5
ln(P(sex=1|species=2) = 0.5) = -0.6931471805599453

P(sex=1|species=3) = 0.48760330578512395
ln(P(sex=1|species=3) = 0.48760330578512395) = -0.7182531016910216

P(island=2|species=1) = 0.3221476510067114
ln(P(island=2|species=1) = 0.3221476510067114) = -1.1327452950375683

P(island=2|species=2) = 0.014084507042253521
ln(P(island=2|species=2) = 0.014084507042253521) = -4.2626798770413155

P(island=2|species=3) = 0.00819672131147541
ln(P(island=2|species=3) = 0.00819672131147541) = -4.804021044733257



{1: 0.3221476510067114, 2: 0.014084507042253521, 3: 0.00819672131147541}

5. Calculate the aposteriori probabilities of the labels and decide which label will Naive Bayes predict for the instance. Use only the logarithm values.

In [471]:
# solution here
#P(sex=1|species=1)P(island=2|species=1)P(species=1)
#P(sex=1|species=2)P(island=2|species=2)P(species=2)
#P(sex=1|species=3)P(island=2|species=3)P(species=3)

p_sex = calculate_conditional_probability(penguin_dataset,"species",{ "sex" : 1},1)
p_island = calculate_conditional_probability(penguin_dataset,"species",{ "island" : 2},1)
# print(p_sex)
# print(p_island)

feature_vals, nr_feature_vals = np.unique(penguin_dataset["species"], return_counts=True)
p_species = []
for specie_nr in nr_feature_vals:
    p_species.append(specie_nr / sum(nr_feature_vals))

# print(nr_feature_vals)
# print(p_species)
max_label = -1
max_prob = -9999999999
for val in feature_vals:
    prob = np.log(p_sex[val]) + np.log(p_island[val]) + np.log(p_species[val-1])
    print(prob)
    if prob > max_prob:
        max_prob = prob
        max_label = val

print()
print("MAP label: " + str(max_label))


#VERIFICARE
#2 ore sa stau sa realizez ca imi trebuie categorical bayes si nu bernoulli bayes asdasdapsdeoqansoieqpwoe1po23op1231

from sklearn.naive_bayes import CategoricalNB
X = penguin_dataset[["sex", "island"]]
y = penguin_dataset["species"]
cl = CategoricalNB(alpha=1).fit(X, y)

new_instance = pd.DataFrame(
  [(1, 2)],
columns = ["sex","island"])

print(cl.feature_log_prob_)
print(cl.predict(new_instance))
print(cl.predict_proba(new_instance))

#print(np.unique(penguin_dataset[penguin_dataset["species"] == 3].filter(items=["island"]),return_counts=True))




P(sex=1|species=1) = 0.5
ln(P(sex=1|species=1) = 0.5) = -0.6931471805599453

P(sex=1|species=2) = 0.5
ln(P(sex=1|species=2) = 0.5) = -0.6931471805599453

P(sex=1|species=3) = 0.48760330578512395
ln(P(sex=1|species=3) = 0.48760330578512395) = -0.7182531016910216

P(island=2|species=1) = 0.3221476510067114
ln(P(island=2|species=1) = 0.3221476510067114) = -1.1327452950375683

P(island=2|species=2) = 0.014084507042253521
ln(P(island=2|species=2) = 0.014084507042253521) = -4.2626798770413155

P(island=2|species=3) = 0.00819672131147541
ln(P(island=2|species=3) = 0.00819672131147541) = -4.804021044733257

-2.6504283438696206
-6.544461842405598
-6.551293143293193

MAP label: 1
[array([[-0.69314718, -0.69314718],
       [-0.69314718, -0.69314718],
       [-0.66865616, -0.7182531 ]]), array([[-1.19728382, -0.97859462, -1.1327453 ],
       [-4.26267988, -0.02857337, -4.26267988],
       [-0.0165293 , -4.80402104, -4.80402104]])]
[1]
[[0.96099563 0.0195688  0.01943557]]


6. **Naive Bayes implemenation**: write a function called `naive_bayes` that takes three arguments:
- `df`: the dataframe which will be used for training
- `index_target`: the index of the column associated with the target feature
- `alpha`: the parameter used for Laplace Smoothing

The function should return a dictionary with the following fields:
- `log_prior`: the logarithmic values of the prior probabilities (the probability of the labels)
- `log_likelihoods`: a n x m x t array, where n - the number of features; m - the number of labels; t - the number of values for a feature; this array will contain the logarithmic values of the likelihoods (P(feature = value | target_feature = label))
- `n_classes`: the number of labels
- `n_feature_classes`: a vector that contains the number of unique values for each attribute
- `classes`: the name of the labels (the values of the target feature).

In [472]:
def naive_bayes(df, index_target = -1, alpha = 1e-10, display_vals = False):
    result_dictionary = {}
    target_feature = df.columns[index_target]
    features = []
    for column in df.columns:
        if column != target_feature:
            features.append(column)
    tf_vals, nr_tf_vals = np.unique(df[target_feature], return_counts=True)
    result_dictionary["n_classes"] = len(tf_vals)
    result_dictionary["classes"] = list(tf_vals)
    p_tf = []
    for tf_val in nr_tf_vals:
        p_tf.append(tf_val / sum(nr_tf_vals))
    p_tf_ln = []
    for i in range(len(p_tf)):
        p_tf_ln.append(np.log(p_tf[i]))
    dict = {}
    for i in range(len(tf_vals)):
        dict[tf_vals[i]] = p_tf_ln[i]
    result_dictionary["log_prior"] = dict
    wrong = 0
    log_likelihoods = []
    p_fs = []
    n_feature_classes = []
    for feature in features:
        f_vals, nr_f_vals = np.unique(df[feature], return_counts=True)
        vect = []
        for f_val in f_vals:
            vect.append(f_val)
        n_feature_classes.append(vect)
    result_dictionary["n_features_classes"] = n_feature_classes
    #calculate conditional entropies returneaza un dictionar de format {valoare_target_feature1 : probabilitate1, valoare_target_feature2 : probabilitate2, ...} pentru o valoare feature
    #aici asociez valoarea in sine a unui feature dictionarul pomenit mai sus si dupa creez o matrice
    #am facut asta stiind ca un vector din matrice ii corespunde unui feature
    for feature in features:
        f_vals, nr_f_vals = np.unique(df[feature], return_counts=True)
        p_f = {}
        vect = []
        for f_val in f_vals:
            p_f[f_val] = calculate_conditional_probability(df, target_feature, {feature : f_val}, alpha, False)
        vect.append(p_f)
        p_fs.append(vect)
    if display_vals:
        print(p_fs)
        print()
    # formatare cum se cere in enunt sau mai bine zis cum afiseaza categoricalNB pentru atributul feature_log_prob_
    for p_f in p_fs:
        vect2 = []
        for tf_val in tf_vals:
            vect = []
            for key,value in p_f[0].items():
                vect.append(np.log(p_f[0][key][tf_val]))
            vect2.append(vect)
        log_likelihoods.append(vect2)
    #nu e necesar dar am vrut sa ma verific imediat sa vad daca am muncit degeaba LOL XD :DDDD
    for index, row in df.iterrows():
        p_max = -99999999
        max_tf_val = -1
        all_cond_p = []
        for feature in features:
            all_cond_p.append(calculate_conditional_probability(df, target_feature, {feature: df.at[index, feature]}, alpha, False))
        p = 1
        for tf_val in tf_vals:
            p = 1
            for cond_p in all_cond_p:
                p *= cond_p[tf_val]
            p *= p_tf[tf_val-1]
            if p > p_max:
                p_max = p
                max_tf_val = tf_val
        if max_tf_val != df.at[index, target_feature]:
            wrong += 1
    result_dictionary["log_likelihood"] = log_likelihoods
    if display_vals:
        print("Accuracy: " + str((len(df.index)-wrong) / len(df.index)))
        print("Accuracy sklearn:", cl.score(df[features], df[target_feature]))
        print()
    return(result_dictionary)

print(naive_bayes(discrete_penguin_dataset, 2, 1, True))

[[{0: {1: 0.5, 2: 0.5, 3: 0.512396694214876}, 1: {1: 0.5, 2: 0.5, 3: 0.48760330578512395}}], [{0: {1: 0.30201342281879195, 2: 0.014084507042253521, 3: 0.9836065573770492}, 1: {1: 0.37583892617449666, 2: 0.971830985915493, 3: 0.00819672131147541}, 2: {1: 0.3221476510067114, 2: 0.014084507042253521, 3: 0.00819672131147541}}]]

Accuracy: 0.7027027027027027
Accuracy sklearn: 0.7027027027027027

{'n_classes': 3, 'classes': [1, 2, 3], 'log_prior': {1: -0.8245358682721073, 2: -1.5886347848043372, 3: -1.0290189968689145}, 'n_features_classes': [[0, 1], [0, 1, 2]], 'log_likelihood': [[[-0.6931471805599453, -0.6931471805599453], [-0.6931471805599453, -0.6931471805599453], [-0.6686561605516496, -0.7182531016910216]], [[-1.1972838161751393, -0.9785946152103099, -1.1327452950375683], [-4.2626798770413155, -0.028573372444056, -4.2626798770413155], [-0.016529301951210582, -4.804021044733257, -4.804021044733257]]]}


7. Train the discrete penguin dataset using your version of Naive Bayes and sklearn's. Compare the values of the parameters.(Be careful at what type of Naive Bayes classifier you pick from sklearn!)

In [473]:
# solution here
result_dictionary = naive_bayes(discrete_penguin_dataset, 2, 1)
df = discrete_penguin_dataset
target_feature = "species"
alpha = 0
features = []
for column in df.columns:
    if column != target_feature:
        features.append(column)
wrong = 0
for index, row in df.iterrows():
        p_max = -99999999
        max_tf_val = -1
        all_cond_p = []
        for class_ in result_dictionary["classes"]:
            p = 0
            for i in range(len(result_dictionary["n_features_classes"])):
                p += result_dictionary["log_likelihood"][i][class_-1][df.at[index, features[i]]]
            p += result_dictionary["log_prior"][class_]
            if p > p_max:
                p_max = p
                max_tf_val = class_
        if max_tf_val != df.at[index, target_feature]:
            wrong += 1
print("Accuracy: " + str((len(df.index)-wrong) / len(df.index)))
print("Accuracy sklearn:", cl.score(df[features], df[target_feature]))


Accuracy: 0.7027027027027027
Accuracy sklearn: 0.7027027027027027


8. Create a function named `nb_predict_prob` that uses the log probabilities calculated by Naive Bayes to infer the aposteriori probability of a new instance `X`.

In [474]:
print(result_dictionary)
def nb_predict_prob(nb_dict, X, use_log = False):
    result_dictionary = nb_dict
    all_p = []
    for class_ in result_dictionary["classes"]:
        p = 1
        for i in range(len(result_dictionary["n_features_classes"])):
            p += result_dictionary["log_likelihood"][i][class_-1][int(X.iloc[[0]].iloc[:, i])]
        p += result_dictionary["log_prior"][class_]
        all_p.append(math.pow(math.e, p)) # formula folosita pt a obtine pondere mai mare pt valoare mai mica
    predict_proba_total = sum(all_p)
    predict_proba = []
    for p in all_p:
        predict_proba.append(p / predict_proba_total)
    return predict_proba

new_instance = pd.DataFrame(
  [(1, 2)],
columns = ["sex","island"])

print()
print(nb_predict_prob(result_dictionary, new_instance))
    
    

{'n_classes': 3, 'classes': [1, 2, 3], 'log_prior': {1: -0.8245358682721073, 2: -1.5886347848043372, 3: -1.0290189968689145}, 'n_features_classes': [[0, 1], [0, 1, 2]], 'log_likelihood': [[[-0.6931471805599453, -0.6931471805599453], [-0.6931471805599453, -0.6931471805599453], [-0.6686561605516496, -0.7182531016910216]], [[-1.1972838161751393, -0.9785946152103099, -1.1327452950375683], [-4.2626798770413155, -0.028573372444056, -4.2626798770413155], [-0.016529301951210582, -4.804021044733257, -4.804021044733257]]]}

[0.960995628673782, 0.019568798053176174, 0.01943557327304176]


9. Create a function that does the Naive Bayes prediction using the Maximum Aposteriori Probability.

In [475]:
def nb_predict(nb_dict, X):
    result_dictionary = nb_dict
    p_max = -99999999
    max_tf_val = -1
    all_cond_p = []
    for class_ in result_dictionary["classes"]:
        p = 0
        for i in range(len(result_dictionary["n_features_classes"])):
            p += result_dictionary["log_likelihood"][i][class_-1][int(X.iloc[[0]].iloc[:, i])]
        p += result_dictionary["log_prior"][class_]
        if p > p_max:
            p_max = p
            max_tf_val = class_
    return max_tf_val

new_instance = pd.DataFrame(
  [(1, 2)],
columns = ["sex","island"])
print(nb_predict(result_dictionary, new_instance))

1


10. Create a function that calculate the accuracy of the trained model on a set of instances `X` with known labels `y`.

In [476]:
def nb_score(nb_dict, X, y):
    wrong = 0
    for i in range(len(X.index)):
        y_label = int(y.iloc[[i]])
        y_pred = nb_predict(nb_dict, X.iloc[[i]])
        if y_label != y_pred:
            wrong += 1
    return (len(X.index)-wrong) / len(X.index)

11. Calculate the training accuracy of your Naive Bayes algorithm. Explain the results.

In [477]:
# solution here
# Exista date inconsistente in tabel deci acuratatea nu poate fi 100%
print(nb_score(result_dictionary, discrete_penguin_dataset[features], discrete_penguin_dataset[target_feature]))

0.7027027027027027


12. Find and calculate all the conditional probabilities (also names likelihoods) used to predict the label for the instance `{"sex" : 1, "island" : 2}` in Joint Bayes. (*Hint*: panda's query function might provide itself useful.)

In [478]:
# solution here
#P(sex=1,island=2|species=1)
#P(sex=1,island=2|species=2)
#P(sex=1,island=2|species=3)

def calculate_conditional_probability_JB(dataset, feature, target_feature, print_res = True):
    feature_vals, nr_feature_vals = np.unique(dataset[feature], return_counts=True)
    p_f_vals = []
    for i in range(len(feature_vals)):
        p_f_vals.append(nr_feature_vals[i] / sum(nr_feature_vals))
    query = ""
    for key, value in target_feature.items():
        query += key + "==" + str(value) + " and "
    query = query[:-5]
    print(query)
    dict = {}
    for feature_val in feature_vals:
        temp_dataset1 = dataset[dataset[feature] == feature_val]
        temp_dataset2 = temp_dataset1.query(query)
        nr_rows = len(temp_dataset2.index)
        dict[feature_val] = nr_rows / len(temp_dataset1.index)
    return dict
        


calculate_conditional_probability_JB(discrete_penguin_dataset,"species",{ "sex" : 1, "island" : 2})
# df = pd.DataFrame({'A': range(1, 6),
#                    'B': range(10, 0, -2),
#                    'C C': range(10, 5, -1)})
# print(df)
# print(df.query('A < B and A==1'))

sex==1 and island==2


{1: 0.1643835616438356, 2: 0.0, 3: 0.0}

13. Calculate the aposteriori probabilities of the labels and decide which label will Joint Bayes predict for the instance. Use the conditional and prior probabilities (*not* the logarithmic values).

In [479]:
# solution here
p_instance = calculate_conditional_probability_JB(discrete_penguin_dataset,"species",{ "sex" : 1, "island" : 2})
p_max = -999999999
max_label = -1
feature_vals, nr_feature_vals = np.unique(discrete_penguin_dataset["species"], return_counts=True)
p_f_vals = []
for i in range(len(feature_vals)):
    p_f_vals.append(nr_feature_vals[i] / sum(nr_feature_vals))
for key,value in p_instance.items():
    p = value * p_f_vals[key-1]
    if p > p_max:
        p_max = p
        max_label = key
print(max_label)

#verificare cu librarie

sex==1 and island==2
1


14. **Joint Bayes implemenation**: write a function called `joint_bayes` that takes two arguments:
- `df`: the dataframe which will be used for training
- `index_target`: the index of the column associated with the target feature

The function should return a dictionary with the following fields:
- `prior_probs`: the prior probabilities (the probability of the labels)
- `likelihoods`: a n x m array, where n - the number of labels; m - the number of combination between the values of the features; each label will have assigned a list containing the joint probability P(feature_1 = value_1, feature_2 = value_2, ...,  feature_n = value_n | target_feature = label)
- `n_classes`: the number of labels
- `n_feature_classes`: a vector that contains the number of unique values for each attribute
- `classes`: the name of the labels (the values of the target feature).

*Hint*: check the imports from the first cell of this notebook.

In [480]:
def joint_bayes(df, index_target = -1, display = False):
    result_dictionary = {}
    target_feature = df.columns[index_target]
    feature_vals, nr_feature_vals = np.unique(df[target_feature], return_counts=True)
    p_f_vals = []
    for i in range(len(feature_vals)):
        p_f_vals.append(nr_feature_vals[i] / sum(nr_feature_vals))
    result_dictionary["prior_probs"] = p_f_vals
    result_dictionary["classes"] = list(np.unique(df[target_feature]))
    features = []
    for column in df.columns:
        if column != target_feature:
            features.append(column)
    n_feature_classes = []
    for feature in features:
        n_feature_classes.append(list(np.unique(df[feature])))
    result_dictionary["n_features_classes"] = n_feature_classes
    result_dictionary["n_classes"] = len(np.unique(df[target_feature]))

    params = []
    for feature in features:
        params.append(np.unique(df[feature]))
    cartesian_product = list(product(*params))

    cond_prob_jb_params = []
    for i in range(len(cartesian_product)):
        cond_prob_jb_param = {}
        for j in range(len(features)):
            cond_prob_jb_param[features[j]] = cartesian_product[i][j]
        cond_prob_jb_params.append(cond_prob_jb_param)
    
    all_p = []
    for cond_prob_jb_param in cond_prob_jb_params:
        p = calculate_conditional_probability_JB(df, target_feature, cond_prob_jb_param, print_res = False)
        all_p.append(p)
    
    # transformare in formatul in care se cere in enunt :///// 
    likelihoods = []
    for feature_val in feature_vals:
        likelihood = []
        for p in all_p:
            likelihood.append(p[feature_val])
        likelihoods.append(likelihood)


    result_dictionary["likelihoods"] = likelihoods
    if display:
        print(cartesian_product)
        print()
        print(cond_prob_jb_params)
        print()
        print(all_p)
        print()
        print(likelihoods)
    return result_dictionary

print(joint_bayes(discrete_penguin_dataset, 2, True))


sex==0 and island==0
sex==0 and island==1
sex==0 and island==2
sex==1 and island==0
sex==1 and island==1
sex==1 and island==2
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]

[{'sex': 0, 'island': 0}, {'sex': 0, 'island': 1}, {'sex': 0, 'island': 2}, {'sex': 1, 'island': 0}, {'sex': 1, 'island': 1}, {'sex': 1, 'island': 2}]

[{1: 0.1506849315068493, 2: 0.0, 3: 0.5126050420168067}, {1: 0.1917808219178082, 2: 0.5, 3: 0.0}, {1: 0.15753424657534246, 2: 0.0, 3: 0.0}, {1: 0.1506849315068493, 2: 0.0, 3: 0.48739495798319327}, {1: 0.18493150684931506, 2: 0.5, 3: 0.0}, {1: 0.1643835616438356, 2: 0.0, 3: 0.0}]

[[0.1506849315068493, 0.1917808219178082, 0.15753424657534246, 0.1506849315068493, 0.18493150684931506, 0.1643835616438356], [0.0, 0.5, 0.0, 0.0, 0.5, 0.0], [0.5126050420168067, 0.0, 0.0, 0.48739495798319327, 0.0, 0.0]]
{'prior_probs': [0.43843843843843844, 0.2042042042042042, 0.35735735735735735], 'classes': [1, 2, 3], 'n_features_classes': [[0, 1], [0, 1, 2]], 'n_classes': 3, 'likelihoo

15. Train the Joint Bayes algorithm on the discrete penguin dataset. Print the obtained dictionary.

In [481]:
# solution here
print(joint_bayes(discrete_penguin_dataset, 2))

sex==0 and island==0
sex==0 and island==1
sex==0 and island==2
sex==1 and island==0
sex==1 and island==1
sex==1 and island==2
{'prior_probs': [0.43843843843843844, 0.2042042042042042, 0.35735735735735735], 'classes': [1, 2, 3], 'n_features_classes': [[0, 1], [0, 1, 2]], 'n_classes': 3, 'likelihoods': [[0.1506849315068493, 0.1917808219178082, 0.15753424657534246, 0.1506849315068493, 0.18493150684931506, 0.1643835616438356], [0.0, 0.5, 0.0, 0.0, 0.5, 0.0], [0.5126050420168067, 0.0, 0.0, 0.48739495798319327, 0.0, 0.0]]}


16. Similarly to Naive Bayes, write the functions used to predict the aposteriori probabilities, the label and the accuracy of the Joint Bayes algorithm.

In [482]:
def jb_predict_prob(jb_dict, X, display = False):
    # cum ar trebui sa folosesc likelihoods cand el in formatul in care e n-am de unde sa stiu pt ce valori de feature-uri sunt probabilitatile???
    # asumam ca ordinea de la product este consistenta...
    params = []
    for n_feature_class in jb_dict["n_features_classes"]:
        params.append(np.unique(n_feature_class))
    cartesian_product = list(product(*params))

    instance = {}
    for column in X.columns:
        instance[column] = int(X.iloc[[0]][column])

    # caut instance in lista de la product
    i_instance = -1
    for i in range(len(cartesian_product)):
        if cartesian_product[i] == tuple(instance.values()):
            i_instance = i
    
    p_max= -999999999
    all_p = []
    for class_ in jb_dict["classes"]:
        p = 1
        for i in range(len(cartesian_product)):
            p *= jb_dict["likelihoods"][class_-1][i_instance] * jb_dict["prior_probs"][class_-1]
        all_p.append(p)
    
    result = []
    for i in range(len(all_p)):
        result.append(all_p[i] / sum(all_p))
    
    if display:
        print(instance)
        print()
        print(cartesian_product)
        print()
    
    return result

def jb_predict(jb_dict, X):
    prediction = jb_predict_prob(jb_dict, X)
    return np.argmax(prediction) + 1

def jb_score(jb_dict, X, y):
    wrong = 0
    for i in range(len(X.index)):
        y_label = int(y.iloc[[i]])
        y_pred = jb_predict(jb_dict, X.iloc[[i]])
        if y_label != y_pred:
            wrong += 1
    return (len(X.index)-wrong) / len(X.index)


jb_dict = joint_bayes(discrete_penguin_dataset, 2)
new_instance = pd.DataFrame(
  [(1, 2)],
columns = ["sex","island"])
print(jb_predict_prob(jb_dict, new_instance))
print(jb_predict(jb_dict, new_instance))
X = discrete_penguin_dataset[["sex","island"]]
y = discrete_penguin_dataset["species"]
print(jb_score(jb_dict, X, y))

sex==0 and island==0
sex==0 and island==1
sex==0 and island==2
sex==1 and island==0
sex==1 and island==1
sex==1 and island==2
[1.0, 0.0, 0.0]
1
0.7027027027027027



17. Calculate the training accuracy of your Joint Bayes algorithm. 

In [483]:
# solution here
print(jb_score(jb_dict, X, y))

0.7027027027027027


## II. kNN (2 points; 0.15 bonus per week)

For this section we will use the entire `penguin_dataset`.

1. Calculate the Euclidean distance between the test instance `{"sex" : 1, "island" : 2, "bill_length" : 20, "bill_depth" : 40, "flipper_length" : 355, "body_mass" : 855}` and the instances from the dataset. Store the values in an object called `instance_distance`. Print the average distance. (*Hint*: check the norm function from the numpy package.)

In [484]:
# solution here
from numpy import linalg as LA
test_instance = pd.DataFrame(
  [(1, 2, 20, 40, 355, 855)],
columns = ["sex","island","bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"])

test_instance_norm = LA.norm(test_instance.values[0])
print(test_instance_norm)

instance_distance = []
df_no_species = penguin_dataset.drop(columns = ["species"])
for index, row in df_no_species.iterrows():
    line = df_no_species.iloc[[index]]
    norm = LA.norm(np.subtract(line.values[0], test_instance.values[0]))
    instance_distance.append(norm)

print(sum(instance_distance) / len(instance_distance))




926.8521996521343
3356.130258608873


2. Find the `5` nearest neighbours of the test instance.

In [485]:
# solution here
line_index_dict = {}
for i in range(len(df_no_species.index)):
    line_index_dict[i] = LA.norm(np.subtract(df_no_species.iloc[[i]].values[0], test_instance.values[0]))

line_index_dict = {k: v for k, v in sorted(line_index_dict.items(), key=lambda item: item[1])}

print(line_index_dict)

i = 0
result_dataframe = pd.DataFrame(columns = ["sex","island","bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","species"])
for key,value in line_index_dict.items():
    if i == 5:
        break
    result_dataframe.loc[i] = penguin_dataset.iloc[[key]].values[0]
    i += 1

print(result_dataframe)

# verificare daca nu cumva ceea ce am facut da identic cu k-nn... raspunsul este ca da. lol
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
X, y = penguin_dataset[['sex', 'island',"bill_length_mm","bill_depth_mm","flipper_length_mm", "body_mass_g"]], penguin_dataset["species"]
knn.fit(X, y)
knn.predict(test_instance)
knn.predict_proba(test_instance)

{303: 1852.5296677786296, 58: 2002.5142621214961, 52: 2002.7792714126037, 110: 2052.020701649961, 48: 2052.0582009290088, 287: 2052.153941594051, 92: 2052.8267876272466, 98: 2076.5178472625753, 138: 2151.379215759044, 39: 2151.9174263897767, 122: 2201.319243090379, 62: 2201.374654619245, 136: 2201.5134930315553, 118: 2201.680687565751, 96: 2226.8536750312087, 37: 2251.511769900393, 114: 2301.310774319714, 54: 2301.458037853395, 35: 2301.675748232144, 23: 2302.4544230016804, 130: 2325.950689503112, 102: 2326.7007994153437, 106: 2350.7782498568426, 74: 2351.0240322038394, 22: 2351.204299928018, 295: 2351.2197621660125, 7: 2351.574147246903, 2: 2400.525169624347, 321: 2400.8549727128457, 313: 2401.1865691778307, 25: 2401.7239933014785, 271: 2401.772564170055, 120: 2425.726942588551, 281: 2450.36917422661, 289: 2450.474913154591, 27: 2450.874964170959, 88: 2450.9238523462946, 33: 2451.3343815155044, 315: 2475.193642929781, 29: 2475.338150637201, 124: 2475.6727691680094, 113: 2475.754464804

array([[0.8, 0.2, 0. ]])

3. Determine the probabilities of the labels that kNN would assign for this test instance (`k = 5`). Which label has the highest probability?

In [486]:
# solution here
species_vals, nr_species_vals = np.unique(result_dataframe["species"], return_counts=True)
all_species_vals = np.unique(penguin_dataset["species"])
probas = [0 for i in range(len(all_species_vals))]
for i in range(len(all_species_vals)):
    if all_species_vals[i] not in species_vals:
        probas[i] = 0
    else:
        probas[i] = (nr_species_vals[i] / sum(nr_species_vals))
print(probas)
print(np.argmax(probas)+1)

[0.8, 0.2, 0]
1


4. Suppose that kNN gives for each neighbour a weight that is equal to the inverse of the distance. Print the changed probabilities, as well as the label predicted by kNN.

In [487]:
# solution here
def get_value_from_dict_by_index(dict_, index):
    i = 0
    for key,value in dict_.items():
        if i == index:
            return value
        i += 1

#lista cu ponderile noi
species_weight_list = []
for i in range(len(result_dataframe.index)):
    species_weight_dict = {}
    species_weight_dict[result_dataframe.iloc[[i]]["species"].values[0]] = 1 / get_value_from_dict_by_index(line_index_dict, i)
    species_weight_list.append(species_weight_dict)

print(species_weight_list)

species_weight_dict_all = {}
for weight in species_weight_list:
    if list(weight.keys())[0] not in species_weight_dict_all.keys():
        species_weight_dict_all[list(weight.keys())[0]] = weight[list(weight.keys())[0]]
    else:
        species_weight_dict_all[list(weight.keys())[0]] += weight[list(weight.keys())[0]]

print(species_weight_dict_all)

changed_probas = []
for i in range(len(all_species_vals)):
    if all_species_vals[i] not in species_weight_dict_all.keys():
        changed_probas.append(0)
    else:
        changed_probas.append(species_weight_dict_all[all_species_vals[i]] / sum(species_weight_dict_all.values()))

print(changed_probas)
print(np.argmax(changed_probas)+1)

[{2.0: 0.0005398024211936649}, {1.0: 0.0004993722236667537}, {1.0: 0.0004993061463506552}, {1.0: 0.0004873245183130626}, {1.0: 0.00048731561295253686}]
{2.0: 0.0005398024211936649, 1.0: 0.001973318501283008}
[0.7852063478657878, 0.2147936521342122, 0]
1


5. **k-NN implementation**: Create a function `knn_predict_prob` that will take six arguments:
- `df`: the dataframe containing the features and the target feature
- `test_x`: a list with the attributes observed for *one* instance
- `k`: the number of nearest neighbours
- `use_weights`: boolean value that indicates whether to assign weights based on the inverse of the distance or not
- `p`: either an integer, indicating the order of the Minkowski distance (p=2 is the equivalent for Euclidean) or a custom distance function
- `index_target`: the index of the column that contains the labels of the target feature

The function should:
- calculate the distance between `test_x` and the observations from the dataset
- extract the k-nearest neighbours
- calculate the weight for each label
- normalize the weights to become probabilities
- return the probability vector, that will indicate the probability of the instance `test_x` to have the label `i`.

In [488]:
import copy
def knn_predict_prob(df, test_x, k, use_weights = True, p = 2, index_target = -1):

    def get_value_from_dict_by_index(dict_, index):
        i = 0
        for key,value in dict_.items():
            if i == index:
                return value
            i += 1

    df_no_target = df.drop(columns = [df.columns[index_target]])
    line_index_dict = {}
    for i in range(len(df_no_target.index)):
        line = df_no_target.iloc[[i]]
        norm = LA.norm(np.subtract(line.values[0], test_x.values[0]),ord=p)
        line_index_dict[i] = norm

    line_index_dict_copy = copy.deepcopy(line_index_dict)

    line_index_dict = {k: v for k, v in sorted(line_index_dict.items(), key=lambda item: item[1])}

    line_index_sorted_old_distance = {}
    for key,value in line_index_dict.items():
        line_index_sorted_old_distance[key] = line_index_dict_copy[key]
    
    # print(line_index_dict_copy)
    # print(line_index_old_distance)

    i = 0
    result_dataframe = pd.DataFrame(columns = df.columns)
    for key,value in line_index_dict.items():
        if i == k:
            break
        result_dataframe.loc[i] = df.iloc[[key]].values[0]
        i += 1
    
    target_feature = df.columns[index_target]
    if not use_weights:
        species_vals, nr_species_vals = np.unique(result_dataframe[target_feature], return_counts=True)
        all_species_vals = np.unique(df[target_feature])
        probas = [0 for i in range(len(all_species_vals))]
        for i in range(len(all_species_vals)):
            if all_species_vals[i] not in species_vals:
                probas[i] = 0
            else:
                probas[all_species_vals[i]-1] = (nr_species_vals[list(species_vals).index(all_species_vals[i])] / sum(nr_species_vals))
        return probas
    else:
        species_weight_list = []
        for i in range(len(result_dataframe.index)):
            species_weight_dict = {}
            val = 0
            if get_value_from_dict_by_index(line_index_sorted_old_distance, i) == 0:
                val  = 1e-10
            else:
                val = get_value_from_dict_by_index(line_index_sorted_old_distance, i)
            species_weight_dict[result_dataframe.iloc[[i]]["species"].values[0]] = 1 / val
            species_weight_list.append(species_weight_dict)

        #print(species_weight_list)
        
        species_weight_dict_all = {}
        for weight in species_weight_list:
            if list(weight.keys())[0] not in species_weight_dict_all.keys():
                species_weight_dict_all[list(weight.keys())[0]] = weight[list(weight.keys())[0]]
            else:
                species_weight_dict_all[list(weight.keys())[0]] += weight[list(weight.keys())[0]]

        all_species_vals = np.unique(df[target_feature])
        changed_probas = [0 for i in range(len(all_species_vals))]
        for i in range(len(all_species_vals)):
            if all_species_vals[i] not in species_weight_dict_all.keys():
                changed_probas[i] = 0
            else:
                changed_probas[all_species_vals[i]-1] = species_weight_dict_all[all_species_vals[i]] / sum(species_weight_dict_all.values())

        return changed_probas
        
print(knn_predict_prob(penguin_dataset, test_instance, k=5, use_weights = False, p = 2, index_target = 6))

[0.8, 0.2, 0]


6. Write a function that, based on the probabilities calculated above, returns the label with the highest probability.

In [489]:
def knn_predict(df, test_x, k, use_weights = True, p = 2, index_target = -1):
    p = knn_predict_prob(df, test_x, k, use_weights, p, index_target)
    return np.argmax(p)+1

print(knn_predict(penguin_dataset, test_instance, 5, use_weights = False, p = 2, index_target = 6))

1


7. Calculate the probabilities and the predicted label for the instance from exercise 1 using the following configurations:
- `k = 11, unweighted, Euclidean distance`
- `k = 11, weighted, Euclidean distance`
- `k = 11, unweighted, Manhattan distance`
- `k = 11, weighted, Manhattan distance`

Compare your results with the results obtained by the `sklearn` implementation.

In [490]:
# solution here

print("Algoritm personal\n")

print("K=11 unweighted euclidean")
print("Probabilities: ",knn_predict_prob(penguin_dataset, test_instance, 11, use_weights = False, p = 2, index_target = 6))
print("Prediction: ", knn_predict(penguin_dataset, test_instance, 11, use_weights = False, p = 2, index_target = 6))

print()

print("K=11 weighted euclidean")
print("Probabilities: ",knn_predict_prob(penguin_dataset, test_instance, 11, use_weights = True, p = 2, index_target = 6))
print("Prediction: ", knn_predict(penguin_dataset, test_instance, 11, use_weights = True, p = 2, index_target = 6))

print()

print("K=11 unweighted manhattan")
print("Probabilities: ",knn_predict_prob(penguin_dataset, test_instance, 11, use_weights = False, p = 1, index_target = 6))
print("Prediction: ", knn_predict(penguin_dataset, test_instance, 11, use_weights = False, p = 1, index_target = 6))

print()

print("K=11 weighted manhattan")
print("Probabilities: ",knn_predict_prob(penguin_dataset, test_instance, 11, use_weights = True, p = 1, index_target = 6))
print("Prediction: ", knn_predict(penguin_dataset, test_instance, 11, use_weights = True, p = 1, index_target = 6))

print()

print("Algoritm sklearn\n")
from sklearn.neighbors import KNeighborsClassifier
X, y = penguin_dataset[['sex', 'island',"bill_length_mm","bill_depth_mm","flipper_length_mm", "body_mass_g"]], penguin_dataset["species"]

print("K=11 unweighted euclidean")
knn = KNeighborsClassifier(n_neighbors=11, weights='uniform', p=2)
knn.fit(X, y)
print(knn.predict_proba(test_instance))
print(knn.predict(test_instance))

print("K=11 weighted euclidean")
knn = KNeighborsClassifier(n_neighbors=11, weights='distance', p=2)
knn.fit(X, y)
print(knn.predict_proba(test_instance))
print(knn.predict(test_instance))

print("K=11 unweighted manhattan")
knn = KNeighborsClassifier(n_neighbors=11, weights='uniform', p=1)
knn.fit(X, y)
print(knn.predict_proba(test_instance))
print(knn.predict(test_instance))

print("K=11 weighted manhattan")
knn = KNeighborsClassifier(n_neighbors=11, weights='distance', p=1)
knn.fit(X, y)
print(knn.predict_proba(test_instance))
print(knn.predict(test_instance))



Algoritm personal

K=11 unweighted euclidean
Probabilities:  [0.8181818181818182, 0.18181818181818182, 0]
Prediction:  1

K=11 weighted euclidean
Probabilities:  [0.8081241299001586, 0.19187587009984133, 0]
Prediction:  1

K=11 unweighted manhattan
Probabilities:  [0.8181818181818182, 0.18181818181818182, 0]
Prediction:  1

K=11 weighted manhattan
Probabilities:  [0.8094697299455473, 0.19053027005445272, 0]
Prediction:  1

Algoritm sklearn

K=11 unweighted euclidean
[[0.81818182 0.18181818 0.        ]]
[1]
K=11 weighted euclidean
[[0.80812413 0.19187587 0.        ]]
[1]
K=11 unweighted manhattan
[[0.81818182 0.18181818 0.        ]]
[1]
K=11 weighted manhattan
[[0.80946973 0.19053027 0.        ]]
[1]


8. Write a function that calculates the accuracy of the kNN algorithm.

In [491]:
# test y pentru ce?
def knn_score(df, test_x, test_y, k, use_weights = True, p = 2, index_target = -1):
    wrong = 0
    for i in range(len(test_x.index)):
        if knn_predict(df, test_x.iloc[[i]], k, use_weights, p, index_target) != test_y.iloc[[i]].values[0]:
            wrong += 1
    return (len(test_x.index) - wrong) / len(test_x.index)

test_x = penguin_dataset[['sex', 'island',"bill_length_mm","bill_depth_mm","flipper_length_mm", "body_mass_g"]]
test_y = penguin_dataset["species"]
print(knn_score(penguin_dataset, test_x, test_y, 11, use_weights = True, p = 2, index_target = 6))

knn = KNeighborsClassifier(n_neighbors=11, weights='distance', p=2)
knn.fit(test_x, test_y)
print(knn.score(test_x, test_y))



# troubleshooting acuratete... a inceput cu 0.43 si a ajuns mai buna decat cea de la sklearn
# df_no_species = penguin_dataset.drop(columns=['species'])
# line = df_no_species.iloc[[160]]


# knn = KNeighborsClassifier(n_neighbors=5, weights='distance', p=2)
# knn.fit(X, y)

# print(knn_predict_prob(penguin_dataset, line, 5, use_weights = True, p = 2, index_target = 6))
# print(knn.predict_proba(line))

# knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', p=2)
# knn.fit(X, y)

# print(knn.predict_proba(line))




1.0
1.0


9. What is the training accuracy of the unweighted kNN when k varies from 3 to 15? (use only odd numbers). Does adding the weight / changing the distance metric improve the score? Justify.

In [493]:
# solution here
for k in range(3,16,2):
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', p=2)
    knn.fit(test_x, test_y)
    print(str(k) + "-nn fara weight euclidian",knn_score(penguin_dataset, test_x, test_y, k, use_weights = False, p = 2, index_target = 6))
    print(str(k) + "-nn fara weight euclidian sklearn",knn.score(test_x, test_y))
    print(str(k) + "-nn fara weight manhattan",knn_score(penguin_dataset, test_x, test_y, k, use_weights = False, p = 1, index_target = 6))
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', p=1)
    knn.fit(test_x, test_y)
    print(str(k) + "-nn fara weight manhattan sklearn",knn.score(test_x, test_y))

    knn = KNeighborsClassifier(n_neighbors=k, weights='distance', p=2)
    knn.fit(test_x, test_y)
    print(str(k) + "-nn cu weight euclidian",knn_score(penguin_dataset, test_x, test_y, k, use_weights = True, p = 2, index_target = 6))
    print(str(k) + "-nn cu weight euclidian sklearn",knn.score(test_x, test_y))
    print(str(k) + "-nn cu weight manhattan",knn_score(penguin_dataset, test_x, test_y, k, use_weights = True, p = 1, index_target = 6))
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance', p=1)
    knn.fit(test_x, test_y)
    print(str(k) + "-nn cu weight manhattan sklearn",knn.score(test_x, test_y))

# acuratetea creste cand se face knn cu pondere pentru ca punctele mai apropiate in plan capata prioritate.
# exista o diferenta intre distanta manhattan si euclidiana, dar nu este foarte mare. diferenta este modul de calcul a distantei

3-nn fara weight euclidian 0.9159159159159159
3-nn fara weight euclidian sklearn 0.9159159159159159
3-nn fara weight manhattan 0.924924924924925
3-nn fara weight manhattan sklearn 0.924924924924925
3-nn cu weight euclidian 1.0
3-nn cu weight euclidian sklearn 1.0
3-nn cu weight manhattan 1.0
3-nn cu weight manhattan sklearn 1.0
5-nn fara weight euclidian 0.8378378378378378
5-nn fara weight euclidian sklearn 0.8378378378378378
5-nn fara weight manhattan 0.8378378378378378
5-nn fara weight manhattan sklearn 0.8378378378378378
5-nn cu weight euclidian 1.0
5-nn cu weight euclidian sklearn 1.0
5-nn cu weight manhattan 1.0
5-nn cu weight manhattan sklearn 1.0
7-nn fara weight euclidian 0.8438438438438438
7-nn fara weight euclidian sklearn 0.8438438438438438
7-nn fara weight manhattan 0.8498498498498499
7-nn fara weight manhattan sklearn 0.8498498498498499
7-nn cu weight euclidian 1.0
7-nn cu weight euclidian sklearn 1.0
7-nn cu weight manhattan 1.0
7-nn cu weight manhattan sklearn 1.0
9-nn f

## III. AdaBoost (3 points; 0.25 bonus per week)

## IV. Testing the performance (1.5 points; 0.1 bonus per week)

# Exercise grading

| Section | Exercise | Points |
| --- | --- | --- |
| I |	1 | 	0.1 |
| I |	2 | 	0.1 |
| I |	3 | 	0.25 |
| I |	4 | 	0.25 |
| I |	5 | 	0.15 |
| I |	6 | 	0.65 |
| I |	7 | 	0.1 |
| I |	8 | 	0.25 |
| I |	9 | 	0.15 |
| I |	10 | 	0.1 |
| I |	11 | 	0.1 |
| I |	12 | 	0.1 |
| I |	13 | 	0.15 |
| I |	14 | 	0.65 |
| I |	15 | 	0.1 |
| I |	16 | 	0.2 |
| I |	17 | 	0.1 |
|II | 	1 | 	0.2 |
|II | 	2 | 	0.1 |
|II | 	3 | 	0.15 |
|II | 	4 | 	0.2 |
|II | 	5 | 	0.72 |
|II | 	6 | 	0.1 |
|II | 	7 | 	0.28 |
|II | 	8 | 	0.1 |
|II | 	9 | 	0.15 |
